In [4]:
import pandas as pd
import numpy as np
from scipy.stats import bernoulli, binom

### 특수한 이산형 확률분포

- 베르누이 분포

In [3]:
# 기댓값과 분산 계산하기
p = 0.3
q = 1-p
E = p
V = p*q # 분산
print(f'[수기] 해당 분포 기댓값은 {E:.3f}, 분산은 {V:.3f}')

E, V = bernoulli.stats(p)
print(f'[라이브러리] 해당 분포의 기댓값은 {E:.3f} 분산은 {V:.3f}')

[수기] 해당 분포 기댓값은 0.300, 분산은 0.210
[라이브러리] 해당 분포의 기댓값은 0.300 분산은 0.210


- 이항분포

In [6]:
# combination 함수 정의하기
fac = np.math.factorial
def combi(a, b) : 
    result = fac(a)/fac(b)/fac(a-b)
    return result

# 근로자가 내년에 회사에서 일하지 않을 확률이 0.1이라고 추정한 경우,
# 시간제 근로자 3명을 무작위로 뽑았을 때 그 중 한 명이 금년에 회사를 떠날 확률은?
n = 3  # 샘플 개수
x = 1  # 확률 변수(떠날 샘플 개수)
p = 0.1

# 기댓값과 분산 계산하기
E = n*p
v = n*p*(1-p)
print(f'[수기] 해당 분포의 기댓값은 {E:.3f} 분산은 {V:.3f}')

[수기] 해당 분포의 기댓값은 0.300 분산은 0.210
